# Algorithms

## String Compression

Uniquely Encode a string to as short a length as possible then decode it. Feel free to look up
established algorithms to attempt the shortest length possible, but cite the algorithm you have
chosen. We will compare your compression against Run Length Encoding, listed next to each
example is the length of the compressed string using Run Length Encoding.

#### Environment
* Developed with Python 3.5.5, installed via Anaconda

### Thought process
* Don't know of any compression algorithms off the top of my head, will research
* Compression depends on the type of data and purpose. We have random data (dictionary/pattern compression is out) and we can't accept lossy compression.
* 'smaz' had come up in some searches for text compression algorithms, but it is incorporating some dictionary/pattern based compression, so I am not going further with that for now. https://github.com/antirez/smaz/blob/master/smaz.c
* Read: https://marknelson.us/posts/2012/10/09/the-random-compression-challenge-turns-ten.html
* Read: https://en.wikipedia.org/wiki/Run-length_encoding 
    * Haven't seen the phrase, but the concept is simple and intuitive - would be an initial stab at compression. The idea of using double characters to denote a run or repetition of a character is a decent improvement (useful for the random strings, but 0 compression if the characters don't repeat anywhere)
    * A quick check of the guessed results of RLE shows that the first string used 47 characters... weird
    * Going to grab a RLE Python implementation and see if the results match 
    * See 'TEST 1'
* After reading around, compressing random data may not be something that's possible.. truly random anyway. Patterns can be found, but I can't copy and paste a bunch of patterns from existing algorithms and pass it off as my own.
* I could use existing patterns.. and pi exists..

#### Dumb idea - "pi compression"
* Encoding:
    * With PI in binary, represent each 8 bits as a byte. 
    * Find the exact input string within the binary representation. 
    * Return the location of the string and number of bytes to read.
* Decoding:
    * Straightforward!
* Issue:
    * I have a suspicion that as the string gets more complex, the search time is going to approach "I am never going to hire this guy in a million years, and even then the search may take longer".
* Potential mitigation:
    * If there was an evolving index recording where certain patterns or beginnings of patterns existed, that could help the search time. Interesting problem: could you eventually be publicly disclosing source code/passwords?
* Issue:
    * Every additional character in the string reduces the chance of finding the pattern by 2^8(guess), and will slow down the search by some other factor.
    * If the patterns are too complex, could representing the index of the start location in Pi take up more data?
* Potential mitigation:
    * Break up the incoming string into 8 bytes, followed by a remainder, and return a list of locations to read 8 bytes from, followed by the remainder (uncompressed) to be appended to the result.
* Further thinking:
    * Googled 'search in pi', found http://angio.net/pi/whynotpi.html, and the odds of finding any NUMBER in the first 200 million digits of pi.
* This isn't worth implementing now , even for a demo on small strings
* Maybe you could combine partial pattern matching in Pi digits with RLE compression on the masked regions?

### More thoughts
* Research led to: https://en.wikipedia.org/wiki/Burrows%E2%80%93Wheeler_transform and https://en.wikipedia.org/wiki/Lyndon_word
* Going to test the BWT with RLE
* See 'Test 2', results weren't good, BUT:
* It's not guaranteed that the data is always random - I can't guarantee random or not - can assume neither..
* Found http://www.data-compression.info/Algorithms/BWT/, very cool site but a lot of dead links. There was mention of a "suffix sorting algorithm" for BWT. 

#### Prerequisite imports and declarations

In [1]:
import typing

try:
    from typecheck_magic import typecheck
except:
    raise Exception("Can't import the typecheck magic for Jupyter")
try:
    import mypy
except ImportError:
    raise Exception("mypy can't be found, which is required by the typecheck magic, please install it")

#### The solution
* Attempt None/RLE/BWT+RLE and return a flag+result for the best compressed data. I don't know anything aobut the data structure going into this compression. I had considered a function that analysed the input BEFORE attempting compression, but that is a rabbit-hole.
* Early stage of the rabbit-hole: 
    * Check how many unique values.. if close to the length, then just return the original string
    * If there are fewer unique values, compress
        * Randomly sample log(n) characters in n-length string, take substrings of length 3 to test for repeats
        * if few repeats found, attempt BWT and retry
* Considered using Move To Front transform, but at this point I am in new territory and jumping between ideas that sound good - making implementation slower. Leaving MTF for another time. Heard mention of bijective BWT, another tempting avenue, but have to stop looking at this!
    
###### Based on
* RLE: https://ieeexplore.ieee.org/document/1447423/
* BWT: http://www.hpl.hp.com/techreports/Compaq-DEC/SRC-RR-124.pdf (https://marknelson.us/posts/1996/09/01/bwt.html)

###### Improvements
* Use binary flags to encode what steps were used
    * Use the order of the flags to represent the order of the transformations
* Include MoveToFront
* Potentially include some analysis of the input string for entropy/length and encode using a chain of approaches (which may be toggled following analysis). 
    * Encode the methods used in a header of the result so result can be decoded correctly. For very large input, would consider sampling the data to help determine compression approach.

In [12]:
from itertools import groupby

FLAG_RLE = "\000"
FLAG_RLE_BWT = "\001"


def bwt(s: str) -> str:
    """Apply Burrows-Wheeler transform to input string."""
    assert "\002" not in s and "\003" not in s, "Input string cannot contain STX and ETX characters"
    s = "\002" + s + "\003"  # Add start and end of text marker
    table = sorted(s[i:] + s[:i] for i in range(len(s)))  # Table of rotations of string
    last_column = [row[-1:] for row in table]  # Last characters of each row
    return "".join(last_column)  # Convert list of characters into string


def ibwt(r: str) -> str:
    """Apply inverse Burrows-Wheeler transform."""
    table = [""] * len(r)  # Make empty table
    for i in range(len(r)):
        table = sorted(r[i] + table[i] for i in range(len(r)))  # Add a column of r
    s = [row for row in table if row.endswith("\003")][0]  # Find the correct row (ending in ETX)
    return s.rstrip("\003").strip("\002")  # Get rid of start and end markers


def rle_enc(string: str) -> str:
    # Returns CHARS|COUNTS
    # Return it this way because:
    #    WW1BBB -> W21B3 -> WWWW...WWWWBBB
    # CHARS|COUNTS:
    #    WW1BBB -> W1B213 -> WW1BBB
    rle_str = rle_cts = ""
    for c, cgen in groupby(string):
        counts = len(list(cgen))
        while counts > 0:
            rle_str += (c)
            rle_cts += chr(counts % 255)
            counts -= 255
    return rle_str+rle_cts


def rle_dec(string: str) -> str:
    half_string = int(len(string)/2)
    decstr = ""
    rle_str, rle_cts = string[:half_string], string[half_string:]
    for c, chr_counts in zip(rle_str, rle_cts):
        counts = ord(chr_counts)
        decstr += c * counts
    return decstr


def encode(string: str) -> str:
    # We prepend a flag to our strings to represent how they were compressed
    # If the string starts with any of our flags, then we have to use a method with a flag
    # Otherwise, we would need a flag to represent non-compressed data, which increases the length!
    can_return_minimal = not (string.startswith(FLAG_RLE) or string.startswith(FLAG_RLE_BWT))
    # Get RLE and RLE+BWT, and prepend a flag for each so we know how to undo it
    rle = FLAG_RLE + rle_enc(string)
    rle_bwt = FLAG_RLE_BWT + rle_enc(bwt(string))
    # Now get all the strings together
    strings = [rle, rle_bwt]
    if can_return_minimal:
        strings.append(string)
    # Sort by length and return the shortest
    strings.sort(key = lambda s: len(s))
    return strings[0]


def decode(string: str) -> str:
    if string.startswith(FLAG_RLE):
        return rle_dec(string[1:])
    elif string.startswith(FLAG_RLE_BWT):
        return ibwt(rle_dec(string[1:]))
    else:
        return string
    

teststr = "BANANAANANANAANANANAANANANAANANANAANANANA"
enc = encode(teststr)
dec = decode(enc)
print("Encoded: %s" % enc)
print("Decoded: %s" % decode(enc))
assert dec == teststr

Encoded: ANBAA
Decoded: BANANAANANANAANANANAANANANAANANANAANANANA


In [7]:
strings = [
    '0DE9MDK9J8I1BMUQ18HARUPOKXFE4HLADWV12OYYTUFI59Y1', # 47
    '6QXXCOLMUNBLYY0WOB5BR2HIR5L5XG02TGRAGV', # 36
    '5PNL', # 4
    'GKF8ANZ2DH6P3B5WWFMELX8XEMRSJGKHMDN932EZTM2O', # 43
    '4ZILNB9DW3Y65GIG4Z5WWICIJN6H7HTU88', # 32
    'Aaaaahhhhhhmmmmmmmuiiiiiiiaaaaaa', # 12
    'WWWWWWWWWWWWBWWWWWWWWWWWWBBBWWWWWWWWWWWWWWWWWWWWWWWWBWWWWWWWWWWWWWW', # 14
    'BANANAANANANAANANANAANANANAANANANAANANANA'
]

for string in strings:
    print("'%s', # %02d" % (string, len(encode(string))))
    assert decode(encode(string)) == string

'0DE9MDK9J8I1BMUQ18HARUPOKXFE4HLADWV12OYYTUFI59Y1', # 48
'6QXXCOLMUNBLYY0WOB5BR2HIR5L5XG02TGRAGV', # 38
'5PNL', # 04
'GKF8ANZ2DH6P3B5WWFMELX8XEMRSJGKHMDN932EZTM2O', # 44
'4ZILNB9DW3Y65GIG4Z5WWICIJN6H7HTU88', # 34
'Aaaaahhhhhhmmmmmmmuiiiiiiiaaaaaa', # 15
'WWWWWWWWWWWWBWWWWWWWWWWWWBBBWWWWWWWWWWWWWWWWWWWWWWWWBWWWWWWWWWWWWWW', # 15
'BANANAANANANAANANANAANANANAANANANAANANANA', # 15


In [13]:
###########
# TEST 1
###########
# Found on rosetta code while looking for a basic RLE implementation, could make one, but trying to save time
#     http://rosettacode.org/wiki/Run-length_encoding#Python
# Interesting - hadn't seen for/else structure before, looked it up, could be handy for cleaner code
# I am getting 16 chars, not 14, for something I definitely know can be compressed using RLE
# Testing an online example gets the same result: https://www.mathcelebrity.com/runlencode.php
# From my understanding, I am pretty sure that 16 characters is the best possible result I can get. 
# This is likely one of the errors that are mentioned in this exercise, so I won't stress about it. 
# Maybe some bytes removed by using full 8bits to represent the count, but if the number in bits is interpreted as a letter then we have a problem..
# Could store the letters in an array of bytes, then the counts in another array of bytes.. 
#   this helps only if there are several large repetitions (>9) of characters. Unlikely. 

def test_encode(input_string):
    count = 1
    prev = ''
    lst = []
    for character in input_string:
        if character != prev:
            if prev:
                entry = (prev,count)
                lst.append(entry)
            count = 1
            prev = character
        else:
            count += 1
    else:
        try:
            entry = (character,count)
            lst.append(entry)
            return (lst, 0)
        except Exception as e:
            print("Exception encountered {e}".format(e=e)) 
            return (e, 1)

res, ecode = test_encode("Aaaaahhhhhhmmmmmmmuiiiiiiiaaaaaa")
resstr = ""
for char, rep_count in res:
    if rep_count > 1:
        resstr += "%s%s%s" % (char, char, rep_count)
    else:
        resstr += char
print("Length[%s], %s" % (len(resstr), resstr))

Length[17], Aaa4hh6mm7uii7aa6


In [16]:
###########
# TEST 2
# Code taken from:
# https://en.wikipedia.org/wiki/Burrows%E2%80%93Wheeler_transform
# Final thoughts: This could be useful, but I can't assume what the data is likely to be composed of. 
# Although some strings are compressable, most seem random.
###########
def bwt(s):
    """Apply Burrows-Wheeler transform to input string."""
    assert "\002" not in s and "\003" not in s, "Input string cannot contain STX and ETX characters"
    s = "\002" + s + "\003"  # Add start and end of text marker
    table = sorted(s[i:] + s[:i] for i in range(len(s)))  # Table of rotations of string
    last_column = [row[-1:] for row in table]  # Last characters of each row
    return "".join(last_column)  # Convert list of characters into string

def ibwt(r):
    """Apply inverse Burrows-Wheeler transform."""
    table = [""] * len(r)  # Make empty table
    for i in range(len(r)):
        table = sorted(r[i] + table[i] for i in range(len(r)))  # Add a column of r
    s = [row for row in table if row.endswith("\003")][0]  # Find the correct row (ending in ETX)
    return s.rstrip("\003").strip("\002")  # Get rid of start and end markers

for teststr in strings:
    r = bwt(teststr)
    print("\nOriginal Length: %s" % len(teststr))
    print("BWT %s --> %s" % (teststr, r))
    res = rle_enc(r)
    print("RLE Length[%s], BWT+RLE Length[%s]" % (len(encode(teststr)), len(res)))
    print("Decoded: %s" % ibwt(decode(res)))
    assert ibwt(rle_dec(res)) == teststr


Original Length: 48
BWT 0DE9MDK9J8I1BMUQ18HARUPOKXFE4HLADWV12OYYTUFI59Y1 --> 1YVQI1EI1JKE5LH10MAFDXU848F9DOH9BP2UUAYTRMWDK9YO
RLE Length[48], BWT+RLE Length[98]
Decoded: KY1POKY1POKY1POKY1POKY1POKY1POKY1POKY1

Original Length: 38
BWT 6QXXCOLMUNBLYY0WOB5BR2HIR5L5XG02TGRAGV --> VGYR0BRLRON5XXTA2H5OBLUWC6BIG2MG0X5QYL
RLE Length[38], BWT+RLE Length[78]
Decoded: COLMUNBL6QY0WOB5BR2HIR5L5XXG02TGRAGV

Original Length: 4
BWT 5PNL --> LNP5
RLE Length[4], BWT+RLE Length[12]
Decoded: 5PNL

Original Length: 44
BWT GKF8ANZ2DH6P3B5WWFMELX8XEMRSJGKHMDN932EZTM2O --> OZ3M9PBHFXN832MMX2KWJDKSGGETHFEDA26MRZW5L8NE
RLE Length[44], BWT+RLE Length[88]
Decoded: FN932EDOKF8AP3B5XENGLZ2DHMTMEM2RWFN932EDO

Original Length: 34
BWT 4ZILNB9DW3Y65GIG4Z5WWICIJN6H7HTU88 --> 8WG6ZYNH8UBNI9I567WGCZIIJLHTDW5344
RLE Length[34], BWT+RLE Length[68]
Decoded: 7ID